M8O-402Б-21 Кириченко Александра

Загрузить данные о перелетах. 

* Рассмотреть методы из лекции об ансамблевых моделях. Оценить разброс характеристики качества для каждого из методов с параметрами по умолчанию, проведя кросс-валидацию по 5ти блокам на тренировочный выборке.
* Выбрать один из методов для дальнейшей работы, демонстрирующий лучшее качество
* Подключить недостающие пакеты, выполнить при необходимости дополнительные преобразования
* Для выбранного метода подобрать лучшие значения гиперпараметров
* Оценить качество настроенного классификатора на тестовой выбоке

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

In [2]:
X_train = pd.read_csv('train_n.csv', sep=',')
y_train = pd.read_csv('train_n_y.csv', sep=',')
y_train = y_train['0']

X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train, test_size=0.3, random_state=12)

In [3]:
models = {
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Кросс-валидация по 5 блокам
best_model_name = None
best_score = 0

for name, model in models.items():
    scores = cross_val_score(model, X_tr, y_tr, cv=5)
    mean_score = scores.mean()
    std_score = scores.std()
    print(f"{name}: mean={mean_score:.4f}, std={std_score:.4f}")
    
    if mean_score > best_score:
        best_score = mean_score
        best_model_name = name

print(f"\nВыбрана модель: {best_model_name}")

Random Forest: mean=0.9607, std=0.0013
Gradient Boosting: mean=0.9423, std=0.0018

Выбрана модель: Random Forest


In [7]:
# Подбор гиперпараметров
model = RandomForestClassifier()
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [1, 5, 10],
    'min_samples_split': [2, 5]
}
#grid_search = GridSearchCV(model, param_grid, cv=5)
#grid_search.fit(X_tr, y_tr)

from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=20, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_tr, y_tr)

print(f"\nЛучшие параметры для {best_model_name}: {random_search.best_params_}")

C:\Users\sasha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 18 is smaller than n_iter=20. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\sasha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



Лучшие параметры для Random Forest: {'n_estimators': 200, 'min_samples_split': 5, 'max_depth': 10}


In [8]:
# Оценка на тестовой выборке
y_pred = random_search.best_estimator_.predict(X_te)
test_acc = accuracy_score(y_te, y_pred)
print(f"Точность на тестовой выборке: {test_acc:.4f}")

Точность на тестовой выборке: 0.9461
